In [302]:
import os

In [303]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction'

In [304]:
os.chdir("../")

In [305]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [306]:
project_link = "C:/Users/RICH-FILES/Desktop/ml/client-subscription-prediction"
os.chdir(project_link)

In [307]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [308]:
project_link = "C:/Users/RICH-FILES/Desktop/ml/client-subscription-prediction"
os.chdir(project_link)

In [309]:
from clientClassifier.constants import *
from clientClassifier.utils.common import read_yaml, create_directories


In [311]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL  = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir    
                
            )
        
        return data_ingestion_config

In [312]:
import os
import urllib.request as request
import zipfile
from clientClassifier import logger
from clientClassifier.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
   
    def download_file(self):
        if not os.path.exists(self.config.local_data_file): 
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_file
                
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
            
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")    
    
    def extract_zip_file(self):
        """ArithmeticError
        zip_file_path:str
        Extracts the zip file to the specified directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)          

In [314]:
# Ensure the working directory is set to the project directory
os.chdir(project_link)

# Creating the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-05 01:09:12,183: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-05 01:09:12,187: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-05 01:09:12,191: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-05 01:09:12,193: INFO: common: created directory at: artifacts]
[2025-04-05 01:09:12,194: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-05 01:09:14,224: INFO: 2582827443: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 66639
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "8ffd57c005bd40ff5aa24c9a6ec24242b8e1b9b480f27baadb4aedacf7521799"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2868:1AC277:25F666:980E66:67F0667D
Accept-Ranges: bytes
Date: Fri